In [13]:
import os
import openai
import json
from Keys import openai_keys
import re
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from sklearn import tree
from sklearn.cluster import KMeans
import joblib
import numpy as np
import pandas as pd

openai.organization = openai_keys['organization']
openai.api_key = openai_keys['api_key']
embedding_model = "text-embedding-ada-002"

def get_embedding(text, model="text-embedding-ada-002"):
   text = text.replace("\n", " ")
   return openai.Embedding.create(input = [text], model=model)['data'][0]['embedding']


In [4]:
# Load the JSON data from the file
with open('Source/reddit-oppenheimer review-14.json', 'r') as json_file:
    comments_by_post = json.load(json_file)

In [6]:
clf = joblib.load('models/embeddingsForest_1.pkl')

In [16]:
class Post:
    def __init__(self, title, analysis, raw, debug=None):
        self.title = title
        self.sentiment = analysis
        self.raw = raw
        self.debug = debug

redditSentiment = list()
iter = 0
for title, comments in comments_by_post.items():
    x = list()
    for com in comments:
        tmp = get_embedding(str(title) + " : " + str(com), model=embedding_model)
        x.append(tmp)

    X = np.array(x).reshape((len(x), -1))

    y = clf.predict(X)
    y_pred = clf.predict_proba(X)
    redditSentiment.append(Post(title, y, comments, y_pred))
    iter += 1
    print(str(iter) + "/" + str(len(comments_by_post)))

1/5
2/5
3/5
4/5
5/5


In [18]:
for sent in redditSentiment:
    for i in range(len(sent.raw)):
        if input(str(sent.raw[i]) + "\nRating: " + str(sent.sentiment[i]) + "\nProbs: " + str(sent.debug[i])) == "stop":
            break